In [1]:
# takes 1h per 100k hands.
# Read acbl_hand_records.pkl file and clean up double dummy (DD) and par errors.
# Best to re-use previous cleaned up results as performance is slow. Use previous results unless record is new.
# It's not sufficient to validate par score vul calcs by simply correcting vul because some scores would be missing.
# Creates ../acbl/acbl_hand_records_cleaned.pkl

# todo:
# acbl_hand_records.pkl are incorrect due to legacy ignoring of vul. Clean here or correct sql in previous step? 
# figure out why this line throws an exception: stdhrs.at[grp_start+handno,'Par'] = pars
# board_record_string has replaced '10' with 'T'. This is confusing as it doesn't match sql. Needed?

# next steps:
# merge-hand-records.ipynb merges acbl and tcg standardized hand records?

# previous steps:
# acbl_sql_to_hand_records.ipynb creates acbl_hand_records.pkl

# Requirements
# https://github.com/Afwas/python-dds with some of my mods.

In [2]:
import pandas as pd
import pathlib
from IPython.display import display # needed to define display() method in vscode
import sys

In [3]:
sys.path.append(str(pathlib.Path.cwd().parent.joinpath('mlBridgeLib')))
sys.path
import mlBridgeLib

In [4]:
# override pandas display options
mlBridgeLib.pd_options_display()

In [5]:
rootPath = pathlib.Path('e:/bridge/data')
acblPath = rootPath.joinpath('acbl')

In [6]:
# takes 20s
acbl_hand_records_filename = 'acbl_hand_records.pkl'
acbl_hand_records_file = acblPath.joinpath(acbl_hand_records_filename)
stdhrs = pd.read_pickle(acbl_hand_records_file)
display(len(stdhrs),stdhrs)

3603864

,ref,hand_record_id,event_id,Board,EventBoard,Dealer,Vul,board_record_string,Hands,Par,DDmakes
0,"(acbl, 2620401)",77908,100961,09,200217A_09,N,E_W,ST6HK32DJ2CAJ9874SQJ942H765DT874CQSA853HQDKQ963CK52SK7HAJT984DA5CT63,"((T6, K32, J2, AJ9874), (A853, Q, KQ963, K52), (K7, AJT984, A5, T63), (QJ942, 765, T874, Q))","(100, [(4, S, *, EW, -1)])","((10, 4, 10, 4, 8), (2, 9, 3, 9, 2), (10, 4, 10, 4, 8), (2, 9, 3, 9, 2))"
1,"(acbl, 2620412)",77908,100961,20,200217A_20,W,Both,ST85HKJDJ43CKJ873SQ32HA643DK96CT94SAJ64HT98DT75CA52SK97HQ752DAQ82CQ6,"((T85, KJ, J43, KJ873), (AJ64, T98, T75, A52), (K97, Q752, AQ82, Q6), (Q32, A643, K96, T94))","(90, [(1, D, , NS, 1), (1, C, , NS, 1), (1, N, , NS, 0)])","((8, 8, 7, 7, 7), (4, 5, 6, 6, 6), (8, 8, 7, 7, 7), (4, 5, 6, 6, 6))"
2,"(acbl, 2620411)",77908,100961,19,200217A_19,S,E_W,STH764DAQ92CQJ653SQJ763HKJTD65CAKTSK984H5DT8743C987SA52HAQ9832DKJC42,"((T, 764, AQ92, QJ653), (K984, 5, T8743, 987), (A52, AQ9832, KJ, 42), (QJ763, KJT, 65, AKT))","(100, [(3, S, *, EW, -1)])","((9, 7, 9, 4, 6), (4, 6, 3, 8, 6), (9, 7, 9, 5, 6), (4, 6, 3, 8, 6))"
3,"(acbl, 2620420)",77908,100961,28,200217A_28,W,N_S,STHQJ9632DJ9CJ932SKQH8DQT86542CT76SJ76HATDAK3CAKQ84SA985432HK754D7C5,"((T, QJ9632, J9, J932), (J76, AT, AK3, AKQ84), (A985432, K754, 7, 5), (KQ, 8, QT86542, T76))","(-300, [(6, H, *, NS, -2)])","((1, 0, 10, 8, 1), (11, 11, 3, 5, 11), (0, 0, 10, 8, 0), (11, 11, 3, 5, 11))"
4,"(acbl, 2620399)",77908,100961,07,200217A_07,S,Both,S2HQT762D92CAKT92SAKJ7HA8DKJ4C8643SQT854HK43DA765CQS963HJ95DQT83CJ75,"((2, QT762, 92, AKT92), (QT854, K43, A765, Q), (963, J95, QT83, J75), (AKJ7, A8, KJ4, 8643))","(-1430, [(6, S, , EW, 0)])","((6, 2, 6, 1, 3), (7, 11, 6, 12, 8), (6, 2, 6, 1, 3), (7, 11, 6, 12, 8))"
5,"(acbl, 2620396)",77908,100961,04,200217A_04,W,Both,S84HAKJ83D4CQT743SA65HQ62DQ8732CK8SJ972HT5DAKJ96C95SKQT3H974DT5CAJ62,"((84, AKJ83, 4, QT743), (J972, T5, AKJ96, 95), (KQT3, 974, T5, AJ62), (A65, Q62, Q8732, K8))","(620, [(4, H, , NS, 0)])","((10, 5, 10, 6, 6), (3, 8, 3, 6, 6), (10, 5, 10, 6, 6), (3, 8, 3, 6, 6))"
6,"(acbl, 2620410)",77908,100961,18,200217A_18,E,N_S,S872HJ8753DQ873CASAJ65HQ4DAKJ4C874S93HAK6D962CQT963SKQT4HT92DT5CKJ52,"((872, J8753, Q873, A), (93, AK6, 962, QT963), (KQT4, T92, T5, KJ52), (AJ65, Q4, AKJ4, 874))","(-120, [(1, N, , EW, 1)])","((4, 4, 5, 5, 5), (9, 8, 7, 7, 8), (4, 5, 5, 5, 5), (9, 8, 7, 7, 8))"
7,"(acbl, 2620400)",77908,100961,08,200217A_08,W,None,S953H98642DJ9CKJ5SAJT6HAJ3DT2CAT96S742HKQ7DK764CQ43SKQ8HT5DAQ853C872,"((953, 98642, J9, KJ5), (742, KQ7, K764, Q43), (KQ8, T5, AQ853, 872), (AJT6, AJ3, T2, AT96))","(-140, [(1, S, , E, 2)])","((4, 6, 6, 4, 5), (8, 7, 7, 9, 8), (4, 6, 6, 4, 5), (8, 7, 7, 8, 8))"
8,"(acbl, 2620397)",77908,100961,05,200217A_05,N,N_S,S98HAKT83D84CAQ82SQJHQJ65DT2CJ7653SAK752H72DAK96CKTST643H94DQJ753C94,"((98, AKT83, 84, AQ82), (AK752, 72, AK96, KT), (T643, 94, QJ753, 94), (QJ, QJ65, T2, J7653))","(-120, [(2, N, , EW, 0)])","((5, 6, 7, 4, 5), (7, 7, 6, 8, 8), (5, 6, 7, 4, 5), (7, 7, 6, 8, 8))"
9,"(acbl, 2620398)",77908,100961,06,200217A_06,E,E_W,SA53HJT8DQ8743CQ9SKQJT87H954DJCA62S94HAQ62DAT965C54S62HK73DK2CKJT873,"((A53, JT8, Q8743, Q9), (94, AQ62, AT965, 54), (62, K73, K2, KJT873), (KQJT87, 954, J, A62))","(-140, [(2, S, , EW, 1)])","((7, 6, 5, 4, 5), (5, 7, 8, 9, 5), (7, 6, 5, 4, 5), (5, 7, 8, 9, 5))"


In [7]:
# todo: what to do with duplicate board_record_string? keep, discard? Keep dups to sustain integrity with board results?
print("unique brs count:",len(stdhrs['board_record_string'].unique()))
print("keep first duplicate brs count:",sum(stdhrs['board_record_string'].duplicated(keep='first')))
print("keep last duplicate brs count:",sum(stdhrs['board_record_string'].duplicated(keep='last')))
print("keep False duplicate brs count:",sum(stdhrs['board_record_string'].duplicated(keep=False)))
stdhrs.sort_values('board_record_string')

unique brs count: 1741626
keep first duplicate brs count: 1862238
keep last duplicate brs count: 1862238
keep False duplicate brs count: 2130749


,ref,hand_record_id,event_id,Board,EventBoard,Dealer,Vul,board_record_string,Hands,Par,DDmakes
2329244,"(acbl, 562562)",16921,19796,20,191010A_20,W,Both,S2H2D543CAKQJT952SAQJ7HAKQT874D8C3SK98H653DKJ76C876ST6543HJ9DAQT92C4,"((2, 2, 543, AKQJT952), (K98, 653, KJ76, 876), (T6543, J9, AQT92, 4), (AQJ7, AKQT874, 8, 3))","(-100, [(6, C, *, NS, -1)])","((11, 9, 2, 4, 2), (2, 3, 11, 9, 3), (11, 9, 2, 4, 2), (2, 3, 11, 8, 2))"
2329208,"(acbl, 562526)",16920,19795,20,191010A_20,W,Both,S2H2D543CAKQJT952SAQJ7HAKQT874D8C3SK98H653DKJ76C876ST6543HJ9DAQT92C4,"((2, 2, 543, AKQJT952), (K98, 653, KJ76, 876), (T6543, J9, AQT92, 4), (AQJ7, AKQT874, 8, 3))","(-100, [(6, C, *, NS, -1)])","((11, 9, 2, 4, 2), (2, 3, 11, 9, 3), (11, 9, 2, 4, 2), (2, 3, 11, 8, 2))"
1713204,"(acbl, 448172)",13494,15657,11,190928A_11,S,None,S2H2D87542CKJ8543SJT974HQJ97DATCQ6SA83HAT6DKQ963CATSKQ65HK8543DJC972,"((2, 2, 87542, KJ8543), (A83, AT6, KQ963, AT), (KQ65, K8543, J, 972), (JT974, QJ97, AT, Q6))","(-430, [(3, N, , W, 1)])","((6, 3, 4, 3, 3), (6, 9, 8, 9, 8), (6, 3, 4, 3, 3), (5, 9, 8, 9, 10))"
196528,"(acbl, 601942)",18107,21157,28,191014A_28,W,N_S,S2H2D876532CQ9853SAT943HQ53DQT94CJS875HAKT974DAKCKTSKQJ6HJ86DJCA7642,"((2, 2, 876532, Q9853), (875, AKT974, AK, KT), (KQJ6, J86, J, A7642), (AT943, Q53, QT94, J))","(-460, [(5, N, , EW, 0)])","((9, 6, 1, 3, 1), (4, 7, 10, 10, 11), (9, 6, 1, 3, 1), (4, 7, 10, 10, 11))"
2774477,"(acbl, 2335944)",69510,88292,32,200203A_32,W,E_W,S2H2D876543CT7432SAT53HKT64DAKJ9CKS864HJ87DQ2CQJ965SKQJ97HAQ953DTCA8,"((2, 2, 876543, T7432), (864, J87, Q2, QJ965), (KQJ97, AQ953, T, A8), (AT53, KT64, AKJ9, K))","(-120, [(2, N, , EW, 0)])","((4, 6, 5, 6, 4), (7, 6, 7, 7, 8), (4, 7, 6, 6, 5), (7, 6, 7, 7, 8))"
2850990,"(acbl, 1208228)",36150,44726,03,191121A_03,S,E_W,S2H2DAJ9654CA8765SAQJT95HAQT953D7CSK874HKJ874DTCT92S63H6DKQ832CKQJ43,"((2, 2, AJ9654, A8765), (K874, KJ874, T, T92), (63, 6, KQ832, KQJ43), (AQJT95, AQT953, 7, ))","(-300, [(7, C, *, NS, -2)])","((11, 10, 0, 0, 1), (1, 2, 12, 12, 2), (11, 10, 0, 0, 1), (1, 2, 12, 12, 2))"
1139996,"(acbl, 6461178)",254594,354737,07,210715A_07,S,Both,S2H2DAKJ9652CA954SJT84HT87DQT73CQ7SQ97HAKJ543DCKJ86SAK653HQ96D84CT32,"((2, 2, AKJ9652, A954), (Q97, AKJ543, , KJ86), (AK653, Q96, 84, T32), (JT84, T87, QT73, Q7))","(120, [(2, N, , NS, 0)])","((8, 9, 4, 7, 8), (4, 3, 8, 5, 3), (8, 9, 4, 7, 8), (4, 3, 8, 6, 4))"
2223403,"(acbl, 6727843)",265244,368064,35,210811A_35,S,E_W,S2H2DAKJ9753CJ542SKJ9HKQJT8DQTCAKTSAT7654HA543D6C73SQ83H976D842CQ986,"((2, 2, AKJ9753, J542), (AT7654, A543, 6, 73), (Q83, 976, 842, Q986), (KJ9, KQJT8, QT, AKT))","(-1100, [(7, D, *, NS, -5)])","((5, 8, 0, 0, 0), (7, 4, 12, 12, 6), (5, 8, 0, 0, 0), (7, 4, 12, 12, 6))"
3288984,"(acbl, 772707)",23224,27683,20,191028A_20,W,Both,S2H2DAKQ3CQJ75432SJ43HAT953DT974C8SKT976HQ6DJ8652C9SAQ85HKJ874DCAKT6,"((2, 2, AKQ3, QJ75432), (KT976, Q6, J8652, 9), (AQ85, KJ874, , AKT6), (J43, AT953, T974, 8))","(990, [(6, N, , NS, 0)])","((12, 8, 9, 8, 12), (0, 5, 3, 5, 1), (12, 8, 9, 8, 12), (0, 5, 3, 5, 1))"
118335,"(acbl, 1026594)",30773,37843,16,191111A_16,W,E_W,S2H2DAKQ8763CK632SKT953HAKQ7D54CT7SQ84HJ93DT92CQ954SAJ76HT8654DJCAJ8,"((2, 2, AKQ8763, K632), (Q84, J93, T92, Q954), (AJ76, T8654, J, AJ8), (KT953, AKQ7, 54, T7))","(400, [(5, D, , NS, 0)])","((10, 11, 7, 5, 9), (2, 1, 5, 8, 2), (10, 11, 7, 5, 9), (2, 1, 5, 7, 2))"


In [8]:
assert len(stdhrs['board_record_string'].unique())+sum(stdhrs['board_record_string'].duplicated(keep='first'))==len(stdhrs)

In [9]:
# mark duplicate board_record_string.
stdhrs['brs_dup'] = stdhrs.duplicated(subset='board_record_string',keep=False)
sum(stdhrs['brs_dup'])

2130749

In [10]:
# experiment with drop_duplicates()
dups = stdhrs['board_record_string'].drop_duplicates()
len(stdhrs),len(dups),len(stdhrs)-len(dups)

(3603864, 1741626, 1862238)

In [11]:
# takes 20s + 1m + 1m
# similar to acbl_hand_records_dd_experiments
acbl_hand_records_cleaned_filename = 'acbl_hand_records_cleaned.pkl'
acbl_hand_records_cleaned_file = acblPath.joinpath(acbl_hand_records_cleaned_filename)
if acbl_hand_records_cleaned_file.exists():
    hrcdf = pd.read_pickle(acbl_hand_records_cleaned_file)
    display(len(hrcdf),hrcdf.head())
    # Preferred to use pd.merge but caused "out of memory" condition. Experiment with using dicts instead.
    DDmakesd = dict(zip(hrcdf['board_record_string'],hrcdf['DDmakes']))
    Pard = dict(zip(hrcdf['board_record_string'],hrcdf['Par']))
    #display(DDmakesd)
    #stdhrs = pd.merge(stdhrs,hrcdf[['EventBoard','DDmakes','Par']],on='EventBoard',how='left')
    #display(hrcdf[hrcdf['EventBoard'].isin(stdhrs['EventBoard'])]
    #display(len(stdhrs),stdhrs)
    # takes 1m
    DDmakesClean = stdhrs.apply(lambda r: DDmakesd[r['board_record_string']] if r['board_record_string'] in DDmakesd else None,axis='columns')
    b = DDmakesClean.notna() & stdhrs['DDmakes'].ne(DDmakesClean)
    assert all(~b)
    # takes 1m
    # verify that dirty Pars are still dirty
    ParClean = stdhrs.apply(lambda r: Pard[r['board_record_string']] if r['board_record_string'] in Pard else None,axis='columns')
    b = ParClean.notna() & stdhrs['Par'].ne(ParClean)
    display(stdhrs[b])
    # assign clean Pars. verify that all Pars are now cleaned.
    stdhrs.loc[b,'Par'] = ParClean
    b = ParClean.notna() & stdhrs['Par'].ne(ParClean)
    assert all(~b)
else:
    ParClean = pd.Series()

3355150

,ref,hand_record_id,event_id,Board,EventBoard,Dealer,Vul,board_record_string,Hands,Par,DDmakes
0,"(acbl, 2620401)",77908,100961,09,200217A_09,N,E_W,ST6HK32DJ2CAJ9874SQJ942H765DT874CQSA853HQDKQ963CK52SK7HAJT984DA5CT63,"((T6, K32, J2, AJ9874), (A853, Q, KQ963, K52), (K7, AJT984, A5, T63), (QJ942, 765, T874, Q))","(200, [(4, S, *, EW, -1)])","((10, 4, 10, 4, 8), (2, 9, 3, 9, 2), (10, 4, 10, 4, 8), (2, 9, 3, 9, 2))"
1,"(acbl, 2620412)",77908,100961,20,200217A_20,W,Both,ST85HKJDJ43CKJ873SQ32HA643DK96CT94SAJ64HT98DT75CA52SK97HQ752DAQ82CQ6,"((T85, KJ, J43, KJ873), (AJ64, T98, T75, A52), (K97, Q752, AQ82, Q6), (Q32, A643, K96, T94))","(90, [(1, D, , NS, 1), (1, C, , NS, 1), (1, N, , NS, 0)])","((8, 8, 7, 7, 7), (4, 5, 6, 6, 6), (8, 8, 7, 7, 7), (4, 5, 6, 6, 6))"
2,"(acbl, 2620411)",77908,100961,19,200217A_19,S,E_W,STH764DAQ92CQJ653SQJ763HKJTD65CAKTSK984H5DT8743C987SA52HAQ9832DKJC42,"((T, 764, AQ92, QJ653), (K984, 5, T8743, 987), (A52, AQ9832, KJ, 42), (QJ763, KJT, 65, AKT))","(140, [(3, H, , NS, 0)])","((9, 7, 9, 4, 6), (4, 6, 3, 8, 6), (9, 7, 9, 5, 6), (4, 6, 3, 8, 6))"
3,"(acbl, 2620420)",77908,100961,28,200217A_28,W,N_S,STHQJ9632DJ9CJ932SKQH8DQT86542CT76SJ76HATDAK3CAKQ84SA985432HK754D7C5,"((T, QJ9632, J9, J932), (J76, AT, AK3, AKQ84), (A985432, K754, 7, 5), (KQ, 8, QT86542, T76))","(-460, [(5, N, , EW, 0)])","((1, 0, 10, 8, 1), (11, 11, 3, 5, 11), (0, 0, 10, 8, 0), (11, 11, 3, 5, 11))"
4,"(acbl, 2620399)",77908,100961,07,200217A_07,S,Both,S2HQT762D92CAKT92SAKJ7HA8DKJ4C8643SQT854HK43DA765CQS963HJ95DQT83CJ75,"((2, QT762, 92, AKT92), (QT854, K43, A765, Q), (963, J95, QT83, J75), (AKJ7, A8, KJ4, 8643))","(-1430, [(6, S, , EW, 0)])","((6, 2, 6, 1, 3), (7, 11, 6, 12, 8), (6, 2, 6, 1, 3), (7, 11, 6, 12, 8))"


,ref,hand_record_id,event_id,Board,EventBoard,Dealer,Vul,board_record_string,Hands,Par,DDmakes,brs_dup
0,"(acbl, 2620401)",77908,100961,09,200217A_09,N,E_W,ST6HK32DJ2CAJ9874SQJ942H765DT874CQSA853HQDKQ963CK52SK7HAJT984DA5CT63,"((T6, K32, J2, AJ9874), (A853, Q, KQ963, K52), (K7, AJT984, A5, T63), (QJ942, 765, T874, Q))","(100, [(4, S, *, EW, -1)])","((10, 4, 10, 4, 8), (2, 9, 3, 9, 2), (10, 4, 10, 4, 8), (2, 9, 3, 9, 2))",False
2,"(acbl, 2620411)",77908,100961,19,200217A_19,S,E_W,STH764DAQ92CQJ653SQJ763HKJTD65CAKTSK984H5DT8743C987SA52HAQ9832DKJC42,"((T, 764, AQ92, QJ653), (K984, 5, T8743, 987), (A52, AQ9832, KJ, 42), (QJ763, KJT, 65, AKT))","(100, [(3, S, *, EW, -1)])","((9, 7, 9, 4, 6), (4, 6, 3, 8, 6), (9, 7, 9, 5, 6), (4, 6, 3, 8, 6))",False
3,"(acbl, 2620420)",77908,100961,28,200217A_28,W,N_S,STHQJ9632DJ9CJ932SKQH8DQT86542CT76SJ76HATDAK3CAKQ84SA985432HK754D7C5,"((T, QJ9632, J9, J932), (J76, AT, AK3, AKQ84), (A985432, K754, 7, 5), (KQ, 8, QT86542, T76))","(-300, [(6, H, *, NS, -2)])","((1, 0, 10, 8, 1), (11, 11, 3, 5, 11), (0, 0, 10, 8, 0), (11, 11, 3, 5, 11))",False
13,"(acbl, 2620404)",77908,100961,12,200217A_12,W,N_S,SAJ63HJ8DAK2CQ765SK54H43DQJ97CAT32S87HK9765D84CKJ94SQT92HAQT2DT653C8,"((AJ63, J8, AK2, Q765), (87, K9765, 84, KJ94), (QT92, AQT2, T653, 8), (K54, 43, QJ97, AT32))","(450, [(4, S, , NS, 1)])","((7, 10, 8, 11, 9), (6, 3, 5, 2, 4), (7, 10, 8, 11, 9), (6, 3, 5, 2, 4))",False
21,"(acbl, 2620395)",77908,100961,03,200217A_03,S,E_W,SK75HK3DKQT852C52SJT63HAT2DJ64CKQTS98HJ9875DA7CA964SAQ42HQ64D93CJ873,"((K75, K3, KQT852, 52), (98, J9875, A7, A964), (AQ42, Q64, 93, J873), (JT63, AT2, J64, KQT))","(100, [(3, H, *, EW, -1)])","((5, 9, 4, 7, 5), (7, 4, 8, 6, 5), (5, 9, 4, 7, 5), (7, 4, 8, 6, 5))",False
36,"(acbl, 2915193)",86634,114736,06,200302A_06,E,E_W,ST62H984DQ6542CKJSQ4HJ765DKJTCQ853SAKJ83H3DA83CA942S975HAKQT2D97CT76,"((T62, 984, Q6542, KJ), (AKJ83, 3, A83, A942), (975, AKQT2, 97, T76), (Q4, J765, KJT, Q853))","(-450, [(4, S, , EW, 1)])","((2, 4, 6, 2, 4), (11, 8, 7, 11, 9), (2, 4, 6, 2, 4), (11, 8, 7, 11, 9))",False
41,"(acbl, 2915212)",86634,114736,25,200302A_25,N,E_W,S643H9843D975CK65SKQ7HQ5DAJ42CAQ93SAJT95HAJ6DKT6C87S82HKT72DQ83CJT42,"((643, 9843, 975, K65), (AJT95, AJ6, KT6, 87), (82, KT72, Q83, JT42), (KQ7, Q5, AJ42, AQ93))","(-990, [(6, N, , EW, 0)])","((2, 1, 3, 1, 1), (11, 12, 9, 12, 12), (2, 1, 3, 1, 1), (10, 12, 9, 12, 12))",False
43,"(acbl, 2915192)",86634,114736,05,200302A_05,N,N_S,S742HKQ743DKQ5C72SA65HT85DJ9CQJT98SKT9HJ2DAT87632C3SQJ83HA96D4CAK654,"((742, KQ743, KQ5, 72), (KT9, J2, AT87632, 3), (QJ83, A96, 4, AK654), (A65, T85, J9, QJT98))","(420, [(4, H, , NS, 0)])","((7, 5, 10, 9, 8), (6, 8, 3, 4, 5), (7, 5, 10, 9, 8), (6, 8, 3, 4, 5))",False
44,"(acbl, 2915199)",86634,114736,12,200302A_12,W,N_S,SAQHKQT74DQ3CJ432SJ765H8DJT854CQT9SK983H952D976CAK7ST42HAJ63DAK2C865,"((AQ, KQT74, Q3, J432), (K983, 952, 976, AK7), (T42, AJ63, AK2, 865), (J765, 8, JT854, QT9))","(420, [(4, H, , NS, 0)])","((9, 6, 10, 6, 9), (4, 7, 3, 7, 4), (8, 6, 10, 6, 9), (4, 7, 3, 7, 4))",False
46,"(acbl, 2915205)",86634,114736,18,200302A_18,E,N_S,SKQT5HAQ4DAKQ532CSJ9863HT63D84CJ87S2HK9875D9CAKT962SA74HJ2DJT76CQ543,"((KQT5, AQ4, AKQ532, ), (2, K9875, 9, AKT962), (A74, J2, JT76, Q543), (J9863, T63, 84, J87))","(1440, [(7, D, , NS, 0)])","((6, 13, 7, 10, 11), (7, 0, 6, 3, 2), (5, 13, 5, 9, 11), (7, 0, 6, 3, 2))",False


In [12]:
# valdiate calculated dd result by comparing against known (assumed) correct dd. Similar to functions.CompareTable().
def CompareDDTables(DDtable1, DDtable2):
    for suit in range(dds.DDS_STRAINS):
        for pl in range(4):
            if DDtable1[suit][pl] != DDtable2[4 * suit + pl]:
                return False
    return True

In [13]:
# valdiate calculated par result by comparing against known (assumed) correct par. Similar to functions.ComparePar().
def ComparePars(par1, par2):
    if par1[0] != par2[0]:
        return False
    if len(par1[1]) != len(par2[1]):
        return False
    for p1,p2 in zip(sorted(par1[1],key=lambda k: k[1]),sorted(par2[1],key=lambda k: k[1])):
        if p1 != p2:
            if p1[1] != p2[1] or p1[2] != p2[2] or p1[3] != p2[3]:
                return False # suit/double/direction difference
            if p1[0]+p1[4] != p2[0]+p2[4]:
                return False # only needs to have trick count same. ignore if levels are same (min level+overs vs max level+0)
    return True

In [14]:
# undertricks: 0 = make; 1-13 = sacrifice
# overTricks: 0-3; e.g. 1 for 4S + 1
# level: 1-7
denom = 'NSHDC'
seats = ['N','E','S','W','NS','EW']

dealer_d = {'N':0, 'E':1, 'S':2, 'W':3}
vul_d = {'None':0, 'Both':1, 'N_S':2, 'E_W':3} # dds vul encoding is weird

In [15]:
# perform initialization for calculation of double dummy (all positions and all strains) and Pars.

import dds
import ctypes
import functions

starting_handno = 0 # for restarting at a particular hando (0-len(stdhrs)). normally 0.

DDdealsPBN = dds.ddTableDealsPBN()
tableRes = dds.ddTablesRes()
pres = dds.allParResults()
presmaster = dds.parResultsMaster()

mode = 0
tFilter = ctypes.c_int * dds.DDS_STRAINS
trumpFilter = tFilter(0, 0, 0, 0, 0)
line = ctypes.create_string_buffer(80)

dds.SetMaxThreads(0)
max_tables = dds.MAXNOOFTABLES
df = stdhrs[ParClean.isna()]
nhands = len(df)
checkpoint_after_every = 1000

Loaded lib <CDLL 'c:\sw\bridge\dds\dds-291\vs2022\x64\Release\dds.dll', handle 7fff6bd90000 at 0x20edd0bd4c0>


In [16]:
# takes 1h per 100k hands.
# Perform dd and par calcs. Restartable.
# todo: implement direction in pbn. always using 'N:' for now.
# todo: some pars in acbl_hand_records.pkl are incorrect due to ignoring of vul. Need to correct pkl in previous step.

for n,grp_start in enumerate(range(starting_handno,nhands,max_tables)):
    n += 1
    grp_count = min(nhands-grp_start,max_tables)
    DDdealsPBN.noOfTables = grp_count
    print(f'Processing group:{n} hands:{grp_start} to {grp_start+grp_count} of {nhands}')
    DDtables = []
    for handno in range(grp_count):
        r = df.iloc[grp_start+handno]
        nsew_tuple = r['Hands']
        pbn = 'N:'+' '.join('.'.join([suit for suit in suits]) for suits in nsew_tuple)
        assert len(pbn) == 1+1+13*4+3*4+3, len(pbn) # 69=='N'+':'+(13 cards per hand)+(3 '.' per suit per hand)+3 spaces
        pbn_encoded = pbn.encode()
        #print(len(pbn),pbn,pbn_encoded)
        DDdealsPBN.deals[handno].cards = pbn_encoded
        nsew_tuple = r['DDmakes']
        DDtable = [nsew_tuple[nesw][cdhsn] for cdhsn in [3,2,1,0,4] for nesw in range(4)] # convert indexing from mlb to dds
        assert len(DDtable) == 5*4, DDtable
        DDtables.append(DDtable)

    # CalcAllTablesPBN will do multi-threading
    res = dds.CalcAllTablesPBN(ctypes.pointer(DDdealsPBN), mode, trumpFilter, ctypes.pointer(tableRes), ctypes.pointer(pres))

    if res != dds.RETURN_NO_FAULT:
        dds.ErrorMessage(res, line)
        print(f'CalcAllTablesPBN: DDS error: {line.value.decode("utf-8")}')
        assert False, grp_start
    
    for handno in range(0, grp_count):
        r = df.iloc[grp_start+handno]
        #display(r)
        ref = r['ref'][1]
        dealer = r['Dealer']
        assert dealer in 'NESW'
        vul = r['Vul']
        assert vul in ['None','N_S','E_W','Both'], r
        par = r['Par']
        assert type(par) is tuple, r
        assert len(par) == 2, r
        assert type(par[0]) is int, r
        assert type(par[1]) is list, r
        assert len(par[1]) > 0, r
        par_result = ctypes.pointer(presmaster)
        dd_result = ctypes.pointer(tableRes.results[handno])

        # Par calculations are not multi-threading
        res = dds.DealerParBin(dd_result, par_result, dealer_d[dealer], vul_d[vul])
        if res != dds.RETURN_NO_FAULT:
            dds.ErrorMessage(res, line)
            print(f'DealerParBin: DDS error: {line.value.decode("utf-8")}')
            assert False, r
        
        score = par_result.contents.score
        if score == 0:
            pars = (0, [(0, '', '', '', 0)]) # par score is for everyone to pass (1 out of 100,000)
        else:
            assert par_result.contents.number > 0, r
            pars = (score,[])
            for i in range(par_result.contents.number):
                ct = par_result.contents.contracts[i]    
                #print(f'Par[{i}]: underTricks:{ct.underTricks} overTricks:{ct.overTricks} level:{ct.level} denom:{ct.denom} seats:{ct.seats}')
                assert ct.underTricks == 0 or ct.overTricks == 0
                pars[1].append((ct.level,denom[ct.denom],'*' if ct.underTricks else '',seats[ct.seats],ct.overTricks-ct.underTricks))

        dd_eq = CompareDDTables(dd_result.contents.resTable, DDtables[handno])
        par_eq = ComparePars(pars, par)
        if not dd_eq or not par_eq:
            functions.PrintPBNHand(f'Group:{n} Hand:{grp_start+handno+1}/{nhands} ACBL Ref:{ref} Vul:{vul}', DDdealsPBN.deals[handno].cards)
            functions.PrintTable(dd_result)
            if not dd_eq:
                print(f'DD mismatch: Current:{DDtables[handno]}')
                # DDtable = dd_result.contents.resTable
                #DDtable = [nsew_tuple[nesw][cdhsn] for cdhsn in [3,2,1,0,4] for nesw in range(4)] # convert indexing from mlb to dds
                # stdhrs.at[grp_start+handno,'DDmakes'] = DDtable
                assert False # temp - need to finish compatible DDtable
            if not par_eq:
                #functions.PrintDealerParBin(par_result)
                if pars[0] != par[0]: # differences in par score are usually due to vul-unknown vs vul-known.
                    print(f'Par score mismatch: Vul ignorance yielding wrong score?: Calculated:{pars[0]}, Current:{par[0]}')
                else:
                    print(f'Par contract mismatch: (Vul ignorance effecting interfering bid?): Calculated:{pars[1]}, Current:{par[1]}')
                    #assert False
                stdhrs.at[r.name,'Par'] = pars
            print()
    if n%checkpoint_after_every == 0: # save every checkpoint_after_every count (1000).
        acbl_hand_records_cleaned_filename = 'acbl_hand_records_cleaned.pkl'
        acbl_hand_records_cleaned_file = acblPath.joinpath(acbl_hand_records_cleaned_filename)
        stdhrs.to_pickle(acbl_hand_records_cleaned_file) # takes 40s
        print(f'Checkpoint at group {n}: df written to {acbl_hand_records_cleaned_file}')
        starting_handno = grp_start
starting_handno = nhands

Processing group:1 hands:0 to 40 of 236705
Processing group:2 hands:40 to 80 of 236705
Processing group:3 hands:80 to 120 of 236705
Processing group:4 hands:120 to 160 of 236705
Processing group:5 hands:160 to 200 of 236705
Processing group:6 hands:200 to 240 of 236705
Processing group:7 hands:240 to 280 of 236705
Processing group:8 hands:280 to 320 of 236705
Group:8 Hand:282/236705 ACBL Ref:499084 Vul:Both
------------------------------------------------                                
            A8754                                                               
            984                                                                 
            K8                                                                  
            T74                                                                 
6                       QJT92                                                   
AQJ5                    K                                                       
6                     

Group:9 Hand:324/236705 ACBL Ref:499146 Vul:Both
------------------------------------------------                                
            A8754                                                               
            984                                                                 
            K8                                                                  
            T74                                                                 
6                       QJT92                                                   
AQJ5                    K                                                       
6                       QJ953                                                   
AKQJ982                 65                                                      
            K3                                                                  
            T7632                                                               
            AT742                                           

Group:10 Hand:378/236705 ACBL Ref:532511 Vul:Both
-------------------------------------------------                               
            2                                                                   
            AJ865                                                               
            AK83                                                                
            J73                                                                 
AQJ76                   T5                                                      
732                     T94                                                     
QJT                     962                                                     
T8                      K9642                                                   
            K9843                                                               
            KQ                                                                  
            754                                            

Group:11 Hand:415/236705 ACBL Ref:667240 Vul:Both
-------------------------------------------------                               
            K7                                                                  
            T76                                                                 
            Q9874                                                               
            AJT                                                                 
QJT                     A965                                                    
AKQ9                    832                                                     
AKJ                     T65                                                     
743                     K98                                                     
            8432                                                                
            J54                                                                 
            32                                             

Processing group:14 hands:520 to 560 of 236705
Processing group:15 hands:560 to 600 of 236705
Processing group:16 hands:600 to 640 of 236705
Processing group:17 hands:640 to 680 of 236705
Processing group:18 hands:680 to 720 of 236705
Processing group:19 hands:720 to 760 of 236705
Processing group:20 hands:760 to 800 of 236705
Processing group:21 hands:800 to 840 of 236705
Processing group:22 hands:840 to 880 of 236705
Processing group:23 hands:880 to 920 of 236705
Processing group:24 hands:920 to 960 of 236705
Processing group:25 hands:960 to 1000 of 236705
Processing group:26 hands:1000 to 1040 of 236705
Processing group:27 hands:1040 to 1080 of 236705
Processing group:28 hands:1080 to 1120 of 236705
Processing group:29 hands:1120 to 1160 of 236705
Processing group:30 hands:1160 to 1200 of 236705
Processing group:31 hands:1200 to 1240 of 236705
Processing group:32 hands:1240 to 1280 of 236705
Processing group:33 hands:1280 to 1320 of 236705
Processing group:34 hands:1320 to 1360 of 2

...

Group:5423 Hand:216883/236705 ACBL Ref:217155 Vul:Both
------------------------------------------------------                          
            KT986                                                               
            A                                                                   
            Q7                                                                  
            AQT73                                                               
Q4                      A532                                                    
T4                      K9762                                                   
T9832                   6                                                       
J864                    K52                                                     
            J7                                                                  
            QJ853                                                               
            AKJ54                                     

Processing group:5498 hands:219880 to 219920 of 236705
Processing group:5499 hands:219920 to 219960 of 236705
Processing group:5500 hands:219960 to 220000 of 236705
Processing group:5501 hands:220000 to 220040 of 236705
Processing group:5502 hands:220040 to 220080 of 236705
Processing group:5503 hands:220080 to 220120 of 236705
Processing group:5504 hands:220120 to 220160 of 236705
Processing group:5505 hands:220160 to 220200 of 236705
Processing group:5506 hands:220200 to 220240 of 236705
Processing group:5507 hands:220240 to 220280 of 236705
Processing group:5508 hands:220280 to 220320 of 236705
Processing group:5509 hands:220320 to 220360 of 236705
Processing group:5510 hands:220360 to 220400 of 236705
Processing group:5511 hands:220400 to 220440 of 236705
Processing group:5512 hands:220440 to 220480 of 236705
Processing group:5513 hands:220480 to 220520 of 236705
Processing group:5514 hands:220520 to 220560 of 236705
Processing group:5515 hands:220560 to 220600 of 236705
Processing

Processing group:5623 hands:224880 to 224920 of 236705
Processing group:5624 hands:224920 to 224960 of 236705
Processing group:5625 hands:224960 to 225000 of 236705
Processing group:5626 hands:225000 to 225040 of 236705
Processing group:5627 hands:225040 to 225080 of 236705
Processing group:5628 hands:225080 to 225120 of 236705
Processing group:5629 hands:225120 to 225160 of 236705
Processing group:5630 hands:225160 to 225200 of 236705
Processing group:5631 hands:225200 to 225240 of 236705
Processing group:5632 hands:225240 to 225280 of 236705
Processing group:5633 hands:225280 to 225320 of 236705
Processing group:5634 hands:225320 to 225360 of 236705
Processing group:5635 hands:225360 to 225400 of 236705
Processing group:5636 hands:225400 to 225440 of 236705
Processing group:5637 hands:225440 to 225480 of 236705
Processing group:5638 hands:225480 to 225520 of 236705
Processing group:5639 hands:225520 to 225560 of 236705
Processing group:5640 hands:225560 to 225600 of 236705
Processing

Processing group:5772 hands:230840 to 230880 of 236705
Processing group:5773 hands:230880 to 230920 of 236705
Processing group:5774 hands:230920 to 230960 of 236705
Processing group:5775 hands:230960 to 231000 of 236705
Processing group:5776 hands:231000 to 231040 of 236705
Processing group:5777 hands:231040 to 231080 of 236705
Processing group:5778 hands:231080 to 231120 of 236705
Processing group:5779 hands:231120 to 231160 of 236705
Processing group:5780 hands:231160 to 231200 of 236705
Processing group:5781 hands:231200 to 231240 of 236705
Processing group:5782 hands:231240 to 231280 of 236705
Processing group:5783 hands:231280 to 231320 of 236705
Processing group:5784 hands:231320 to 231360 of 236705
Processing group:5785 hands:231360 to 231400 of 236705
Processing group:5786 hands:231400 to 231440 of 236705
Processing group:5787 hands:231440 to 231480 of 236705
Processing group:5788 hands:231480 to 231520 of 236705
Processing group:5789 hands:231520 to 231560 of 236705
Processing

Processing group:5809 hands:232320 to 232360 of 236705
Processing group:5810 hands:232360 to 232400 of 236705
Processing group:5811 hands:232400 to 232440 of 236705
Processing group:5812 hands:232440 to 232480 of 236705
Processing group:5813 hands:232480 to 232520 of 236705
Processing group:5814 hands:232520 to 232560 of 236705
Processing group:5815 hands:232560 to 232600 of 236705
Processing group:5816 hands:232600 to 232640 of 236705
Processing group:5817 hands:232640 to 232680 of 236705
Processing group:5818 hands:232680 to 232720 of 236705
Processing group:5819 hands:232720 to 232760 of 236705
Processing group:5820 hands:232760 to 232800 of 236705
Processing group:5821 hands:232800 to 232840 of 236705
Processing group:5822 hands:232840 to 232880 of 236705
Processing group:5823 hands:232880 to 232920 of 236705
Processing group:5824 hands:232920 to 232960 of 236705
Processing group:5825 hands:232960 to 233000 of 236705
Processing group:5826 hands:233000 to 233040 of 236705
Processing

In [17]:
# takes 40s
acbl_hand_records_cleaned_filename = 'acbl_hand_records_cleaned.pkl'
acbl_hand_records_cleaned_file = acblPath.joinpath(acbl_hand_records_cleaned_filename)
stdhrs.to_pickle(acbl_hand_records_cleaned_file)